# [모듈 1.1] Semantic Textual Similarity 모델 파인 튜닝

아래의 노트북은 김대근님의 워크샵을 참조 하였습니다. 
- [sentence-bert-finetuning](https://github.com/daekeun-ml/sm-kornlp-usecases/tree/main/sentence-bert-finetuning)

---


# 0. Introduction
---

본 모듈에서는 문장 임베딩을 산출하는 Sentence-BERT 모델을 STS(Semantic Textual Similarity) 데이터셋으로 파인튜닝해 봅니다.
SentenceTransformers 패키지를 사용하면 파인튜닝을 쉽게 수행할 수 있습니다. 다만, 현 시점에는 분산 훈련 기능 지원이 잘 되지 않으므로, 대용량 데이터셋으로 파인튜닝하는 니즈가 있다면 커스텀 훈련 코드를 직접 작성하셔야 합니다.


### References

- Hugging Face Tutorial: https://huggingface.co/docs/transformers/training
- Sentence-BERT paper: https://arxiv.org/abs/1908.10084
- SentenceTransformers: https://www.sbert.net
- KorNLU Datasets: https://github.com/kakaobrain/KorNLUDatasets


# 1. Setup Environments
---


#### 사용자 정의 라이브러러 환경 셋업 및 라이브러리 로딩

In [23]:
import sys

In [24]:
%load_ext autoreload
%autoreload 2

sys.path.append('src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os
import sys
import json
import logging
import argparse
import torch
import gzip
import csv
import math
import urllib
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm

from datetime import datetime
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from transformers.trainer_utils import get_last_checkpoint

from os.path import exists

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[LoggingHandler()]
)

logger = logging.getLogger(__name__)

# 2. 데이터 준비 및 알고리즘 훈련에 관련된 변수 준비

## 2.1. 데이타 저장 위치 및 모델 훈련 후의 아티펙트 위치 정의 

In [26]:
dataset= 'KorSTS'

train_dir = f'data/{dataset}/train'
valid_dir = f'data/{dataset}/valid'
test_dir = f'data/{dataset}/test'


os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True) 
os.makedirs(test_dir, exist_ok=True) 

chkpt_dir = f'{dataset}/chkpt'
model_dir = f'{dataset}/model'
output_data_dir = f'{dataset}/output'

os.makedirs(chkpt_dir, exist_ok=True) 
os.makedirs(model_dir, exist_ok=True)
os.makedirs(output_data_dir, exist_ok=True) 

# 기존 파일 제거
!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 


## 2.2. 환경 변수 정의
- 아래의 환경 변수는 아래 `입력 변수` 를  정의 할때에 사용합니다.
- "잠시 이런 것이 있다" 하고 지나가시기를 바랍니다.
    - 아래 스크립트는 현재의 "세이지 메이커 노트북" 에서도 동작하고, 추후에 세이지 메이커의 도커 컨테이노를 통한 모델 훈련시에 도 사용하기 위한 코드 입니다. 
    - `os.environ.get('SM_CURRENT_HOST')` 이 NULL 이기에 아래의 환경 변수가 세팅이 됩니다.

In [27]:

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(torch.cuda.device_count())
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_VALID'] = f'{src_dir}/{valid_dir}'
    os.environ['SM_CHANNEL_TEST'] = f'{src_dir}/{test_dir}'



## 2.3. Argument parser 함수 준비 및 변수 로딩
- parser_args() 를 정의하고 정의된 변수를 로딩 하여 값을 확인 합니다.
    - 아래의 코드는 세이지 메이커에서 훈련시에 바로 재사용 합니다.

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 

In [28]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # 알고리즘에 대한 사용자 정의 세팅
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=32)
    parser.add_argument("--warmup_steps", type=int, default=100)
    parser.add_argument("--logging_steps", type=int, default=100)
    parser.add_argument("--learning_rate", type=str, default=5e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--tokenizer_id", type=str, default='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    parser.add_argument("--model_id", type=str, default='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    
    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--train_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--valid_dir", type=str, default=os.environ["SM_CHANNEL_VALID"])
    parser.add_argument("--test_dir", type=str, default=os.environ["SM_CHANNEL_TEST"])    
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args


args = parser_args(train_notebook=True) 

args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****\n")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))



2023-02-26 13:17:58 - ***** Arguments *****

2023-02-26 13:17:58 - epochs=1
seed=42
train_batch_size=32
eval_batch_size=32
warmup_steps=100
logging_steps=100
learning_rate=5e-05
disable_tqdm=False
fp16=True
tokenizer_id=sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
model_id=sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
output_data_dir=/home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/KorSTS/output
model_dir=/home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/KorSTS/model
n_gpus=4
train_dir=/home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/data/KorSTS/train
valid_dir=/home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/data/KorSTS/valid
test_dir=/home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/data/KorSTS/test
chkpt_dir=KorSTS

# 3. Data Preparation
---
본 핸즈온에서 사용할 데이터셋은 KorSTS (https://github.com/kakaobrain/KorNLUDatasets) 와 KLUE-STS (https://github.com/KLUE-benchmark/KLUE) 입니다.
단일 데이터셋으로 훈련해도 무방하지만, 두 데이터셋을 모두 활용하여 훈련 시, 약간의 성능 향상이 있습니다.

### Training Tips
SBERT 훈련은 일반적으로 아래 3가지 방법들을 베이스라인으로 사용합니다.
1. NLI (Natural Language Inference) 데이터셋으로 훈련
2. STS 데이터셋으로 훈련
3. NLI 데이터셋으로 훈련 후 STS 데이터셋으로 파인튜닝

한국어 데이터의 경우, STS의 훈련 데이터가 상대적으로 적음에도 불구하고 NLI 기반 모델보다 예측 성능이 우수합니다. 따라서, 2번째 방법으로 진행합니다. <br>
다만, STS보다 조금 더 좋은 예측 성능을 원한다면 NLI 데이터로 먼저 훈련하고 STS 데이터셋으로 이어서 훈련하는 것을 권장합니다.

아래는 SKT Ko-BERT 로 Korean Sentence BERT 의 벤치 마킹 결과 입니다. 
![sts_benchmark.png](img/sts_benchmark.png)
출처: Korean Sentence BERT,  https://github.com/SKTBrain/KoBERT#korean-sentence-bert

## 3.1. KLUE-STS 데이터셋 다운로드 및 피쳐셋 생성
KLUE-STS 데이터셋을 허깅페이스 데이터셋 허브에서 다운로드 후, SBERT 훈련에 필요한 피쳐셋을 생성합니다.

In [29]:
logger.info("Read KLUE-STS train/dev dataset")
datasets = load_dataset("klue", "sts")

train_samples = []
dev_samples = []

for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화
        inp_example = InputExample(texts=[example["sentence1"], example["sentence2"]], label=score)

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

2023-02-26 13:17:58 - Read KLUE-STS train/dev dataset
2023-02-26 13:18:00 - Reusing dataset klue (/home/ec2-user/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
print("dev_samples: ", len(dev_samples))
print("train_samples: ", len(train_samples))

dev_samples:  519
train_samples:  11668


In [31]:
num = 2
def show_head_tail_samples(num, train_samples ):
    print(f"The first {num} is \n")
    for i in range(num):
        print(train_samples[i].label , train_samples[i].texts)

    print(f"\nThe last {num} is \n")    
    for i in range(len(train_samples) -num, len(train_samples)):
        print(train_samples[i].label , train_samples[i].texts)    
        
show_head_tail_samples(num, train_samples)        

The first 2 is 

0.74 ['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.', '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.']
0.0 ['위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.', '시민들 스스로 자발적인 예방 노력을\xa0한 것은 아산 뿐만이 아니었다.']

The last 2 is 

0.9400000000000001 ['개 1마리 고양이 3마리 너무 귀여워요!', '개 한 마리와 고양이 세 마리는 정말 귀여워요!']
0.6599999999999999 ['학회 홍보 메일은 회신 메일을 보내지마', '학회 홍보 메일은 회신 하지마']



## 3.2. KorSTS 데이터셋 다운로드 및 피쳐셋 생성
KorSTS 데이터셋은 허깅페이스에도 등록되어 있지만, 향후 여러분의 커스텀 데이터셋을 같이 사용하는 유즈케이스를 고려하여 GitHub의 데이터셋을 다운로드받아 사용하겠습니다. 

In [32]:
repo = 'https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS'
sts_train = f'{args.train_dir}/sts-train.tsv'
sts_valid = f'{args.valid_dir}/sts-dev.tsv'
sts_test = f'{args.test_dir}/sts-test.tsv'

if exists(sts_train) and exists(sts_valid) and exists(sts_test):
    logger.info("File exists")
else: 
    logger.info("File is downloaded")
    urllib.request.urlretrieve(f'{repo}/sts-train.tsv', filename=f'{args.train_dir}/sts-train.tsv')
    urllib.request.urlretrieve(f'{repo}/sts-dev.tsv', filename=f'{args.valid_dir}/sts-dev.tsv')
    urllib.request.urlretrieve(f'{repo}/sts-test.tsv', filename=f'{args.test_dir}/sts-test.tsv')

    

2023-02-26 13:18:01 - File exists


In [33]:
logger.info("Read KorSTS train dataset")

with open(f'{args.train_dir}/sts-train.tsv', 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row["sentence1"] and row["sentence2"]:          
            score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
            inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)
            train_samples.append(inp_example)
            
logging.info("Read KorSTS dev dataset")            
with open(f'{args.valid_dir}/sts-dev.tsv', 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row["sentence1"] and row["sentence2"]:        
            score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
            inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)
            dev_samples.append(inp_example)            

2023-02-26 13:18:01 - Read KorSTS train dataset
2023-02-26 13:18:01 - Read KorSTS dev dataset


In [34]:
print("dev_samples: ", len(dev_samples))
print("train_samples: ", len(train_samples))

dev_samples:  2019
train_samples:  17417


In [35]:
show_head_tail_samples(num, train_samples)        

The first 2 is 

0.74 ['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.', '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.']
0.0 ['위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.', '시민들 스스로 자발적인 예방 노력을\xa0한 것은 아산 뿐만이 아니었다.']

The last 2 is 

0.0 ['중국, 인도는 양국 관계를 증진시키겠다고 맹세한다', '중국은 불안한 주식 거래자들을 안심시키기 위해 뒤뚱거리고 있다.']
0.0 ['푸틴 대변인 : 도핑 혐의는 근거 없는 것으로 보인다.', '가장 최근의 심한 날씨 : 토네이도 후 텍사스에서 1명 사망']


# 4. Training
---



## 4.1. Model 준비

- model_name = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
    - https://huggingface.co/sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
        - 모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.    
        - 출처: [08) BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇](https://wikidocs.net/154530)
    - 위 사이트에 Deprecated 되었다고 하지만, 아래에서 훈련후 Kor-STS Test Data 로 0.801 의 유사도 나옴.
- model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v1'     
    - https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models
    - 아래에서 훈련후 Kor-STS Test Data 로 0.831 의 유사도 나옴.

In [36]:
model_name = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
# model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v1'

train_batch_size = args.train_batch_size
num_epochs = args.epochs
model_save_path = f'{args.model_dir}/training_sts_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
logger.info(model_save_path)

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name)

2023-02-26 13:18:02 - /home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/KorSTS/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2023-02-26_13-18-02


문장 임베딩을 계산하기 위한 Pooler를 정의합니다. BERT로 분류 태스크를 수행할 때는 첫 번째 [CLS] 토큰의 출력 벡터를 임베딩 벡터로 사용하지만, SBERT에서는 BERT의 모든 토큰들의 출력 벡터들을 사용하여 임베딩 벡터를 계산합니다. 이 때 mean pooling이나 max pooling을 사용할 수 있으며, 본 예제에서는 mean pooling을 사용합니다.

In [37]:
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

2023-02-26 13:18:07 - Use pytorch device: cuda


## 4.2. 데이터 로더 준비

In [38]:
train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)



## 4.3. Loss 및 evaluator 준비

모델 훈련 및 검증에 필요한 클래스 인스턴스를 생성합니다. 베이스라인으로 사용되는 검증 지표는 두 문장의 임베딩 벡터의 유사도를 산출하는 코사인 유사도입니다.

In [39]:
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

2023-02-26 13:18:08 - Warmup-steps: 55


훈련을 수행합니다. 분산 훈련을 수행하지는 않지만, 데이터 볼륨이 크지 않으므로 수 분 내에 훈련이 완료됩니다.

## 4.4. 훈련 실행

In [40]:
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.5),
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    use_amp=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

2023-02-26 13:18:45 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 272 steps:
2023-02-26 13:18:49 - Cosine-Similarity :	Pearson: 0.8421	Spearman: 0.8423
2023-02-26 13:18:49 - Manhattan-Distance:	Pearson: 0.8339	Spearman: 0.8367
2023-02-26 13:18:49 - Euclidean-Distance:	Pearson: 0.8344	Spearman: 0.8373
2023-02-26 13:18:49 - Dot-Product-Similarity:	Pearson: 0.8076	Spearman: 0.8078
2023-02-26 13:18:49 - Save model to /home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/KorSTS/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2023-02-26_13-18-02
2023-02-26 13:19:29 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 544 steps:
2023-02-26 13:19:32 - Cosine-Similarity :	Pearson: 0.8481	Spearman: 0.8479
2023-02-26 13:19:32 - Manhattan-Distance:	Pearson: 0.8362	Spearman: 0.8400
2023-02-26 13:19:32 - Euclidean-Distance:	Pearson: 0.8367	Spe


# 5. 모델 평가
---
훈련이 완료되었다면, 테스트 데이터셋으로 예측 성능을 볼 수 있는 지표들을 산출합니다.

## 5.1. Test 데이터 준비

In [41]:
test_samples = []
logger.info("Read KorSTS test dataset")            
with open(f'{args.test_dir}/sts-test.tsv', 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row["sentence1"] and row["sentence2"]:        
            score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
            inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)
            test_samples.append(inp_example)        

2023-02-26 13:19:49 - Read KorSTS test dataset


## 5.2. 훈련된 모델 로딩

In [42]:
##############################################################################
# Load the stored model and evaluate its performance on STS benchmark dataset
##############################################################################

print("model_save_path: \n", model_save_path, "\n\n")
model = SentenceTransformer(model_save_path)



model_save_path: 
 /home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/KorSTS/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2023-02-26_13-18-02 


2023-02-26 13:19:49 - Load pretrained SentenceTransformer: /home/ec2-user/SageMaker/NLP-HuggingFace-On-SageMaker/3_Semantic-Textual-Similarity/3_Training/KorSTS/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2023-02-26_13-18-02
2023-02-26 13:19:53 - Use pytorch device: cuda


## 5.3. Evaluator 생성

In [43]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

2023-02-26 13:19:53 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2023-02-26 13:19:55 - Cosine-Similarity :	Pearson: 0.8274	Spearman: 0.8289
2023-02-26 13:19:55 - Manhattan-Distance:	Pearson: 0.8228	Spearman: 0.8268
2023-02-26 13:19:55 - Euclidean-Distance:	Pearson: 0.8231	Spearman: 0.8271
2023-02-26 13:19:55 - Dot-Product-Similarity:	Pearson: 0.7632	Spearman: 0.7607


0.8289243002163424

# 6. 모델 경로 저장

In [44]:
%store model_save_path

Stored 'model_save_path' (str)
